## Install

In [ ]:
pip install pandas

In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

2024-11-04 15:13:34.573238: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
pip install --upgrade accelerate

## Test 1 (Commented out but first test)

In [11]:
#first test run.
# import pandas as pd
# from datasets import Dataset, DatasetDict
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# data = pd.read_csv("data/merged_label.csv")

# data = data[['text', 'label']]

# dataset = Dataset.from_pandas(data)

# train_test_split = dataset.train_test_split(test_size=0.2)

# tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
# model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=2)

# def tokenize_function(examples):
#     return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# tokenized_dataset = train_test_split.map(tokenize_function, batched=True)

# training_args = TrainingArguments(
#     output_dir='./results',
#     eval_strategy="epoch",
#     learning_rate=3e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     logging_steps=500
# )

# def compute_metrics(p):
#     preds = p.predictions.argmax(axis=1)
#     precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
#     acc = accuracy_score(p.label_ids, preds)
#     return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset['train'],
#     eval_dataset=tokenized_dataset['test'],
#     compute_metrics=compute_metrics,
# )

# trainer.train()

# results = trainer.evaluate()
# print(results)

# predictions = trainer.predict(tokenized_dataset['test'])
# pred_labels = predictions.predictions.argmax(axis=1)
# true_labels = predictions.label_ids

# output_df = pd.DataFrame({'Text': tokenized_dataset['test']['text'], 'True Label': true_labels, 'Predicted Label': pred_labels})
# print(output_df)

## SciBERT Test 2 + output to csv

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

data = pd.read_csv("data/merged_label.csv")
data = data[['text', 'label']]
dataset = Dataset.from_pandas(data)

train_test_split = dataset.train_test_split(test_size=0.2)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=2)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# Tokenize the dataset
tokenized_dataset = train_test_split.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=500
)

# Compute metrics
def compute_metrics(p):
    preds = p.predictions.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    
    # Calculate Balanced Error Rate (BER)
    FP = ((preds == 1) & (p.label_ids == 0)).sum()
    FN = ((preds == 0) & (p.label_ids == 1)).sum()
    P = (p.label_ids == 1).sum()  # Positive samples
    N = (p.label_ids == 0).sum()  # Negative samples
    
    # Avoid division by zero
    if P + N == 0:
        ber = None
    else:
        ber = (FP / P + FN / N) / 2 if P > 0 and N > 0 else None
    
    return {"accuracy": acc, "recall": recall, "ber": ber}

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Make predictions
predictions = trainer.predict(tokenized_dataset['test'])
pred_labels = predictions.predictions.argmax(axis=1)
true_labels = predictions.label_ids

# Prepare output DataFrame
output_df = pd.DataFrame({
    'model_name': ['scibert'],  # Change this to your model name
    'BER': [results['eval_ber']],
    'recall': [results['eval_recall']],
    'accuracy': [results['eval_accuracy']]
})

# Save to CSV
output_df.to_csv('../data/model_results/scibert.csv', index=False)

print(output_df)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/105 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Recall,Ber
1,No log,0.617458,0.740741,0.846154,0.263736
2,No log,0.564529,0.703704,0.692308,0.296703
3,No log,0.606332,0.703704,0.769231,0.299451


{'eval_loss': 0.606332004070282, 'eval_accuracy': 0.7037037037037037, 'eval_recall': 0.7692307692307693, 'eval_ber': 0.29945054945054944, 'eval_runtime': 12.6439, 'eval_samples_per_second': 2.135, 'eval_steps_per_second': 0.158, 'epoch': 3.0}
  model_name       BER    recall  accuracy
0    scibert  0.299451  0.769231  0.703704
